
# **Programming Assignment 1 - Semantic Segmentation**

#### **Professor**: Dário Oliveira  
#### **Monitor**: João Alcindo

Neste **Programming Assignment (PA)**, você irá trabalhar com a tarefa de **segmentação semântica**, utilizando uma das arquiteturas vistas em aula: a **U-Net**. O objetivo principal é aplicar a rede para segmentação de imagens e ajustar parâmetros para obter melhores resultados. O dataset que deverá ser utilizado pode ser encontrado no seguinte link: [DATASET](https://drive.google.com/file/d/1WUX4z6c7ayJz-NwChkYiybh7WSEpEoDH/view?usp=sharing).


![](https://drive.google.com/uc?export=view&id=1JNBM514DlNycUBwH08PSlqKUqOwhk2Qv)



### **Instruções:**

1. **Escolha do Ambiente de Execução**:  
   Você pode optar por utilizar o **Google Colab** ou o **Kaggle Notebook** como ambiente de desenvolvimento. Certifique-se de configurar o ambiente adequadamente, instalando as bibliotecas necessárias.

2. **Download dos Dados**:  
   Baixe o dataset fornecido e faça a organização adequada das pastas e arquivos para facilitar o carregamento durante a execução do código.

3. **Criação de um Dataset Customizado**:  
   As máscaras presentes no dataset estão no formato **RGB**. Você precisará convertê-las para uma matriz de **labels**. Existe um arquivo CSV disponível contendo as informações sobre a correspondência entre cores e rótulos. Utilize essas informações para realizar a conversão corretamente.

4. **Construção da Arquitetura U-Net**:  
   Implemente a U-Net para a tarefa de segmentação. Você pode usar a estrutura básica apresentada em aula ou fazer modificações.

   ![Estrutura U-Net](https://camo.githubusercontent.com/6b548ee09b97874014d72903c891360beb0989e74b4585249436421558faa89d/68747470733a2f2f692e696d6775722e636f6d2f6a6544567071462e706e67)

5. **Experimentação com Diferentes U-Nets**:  
   A U-Net pode ter diferentes capacidades dependendo da sua profundidade. Na imagem acima, o encoder tem cinco blocos convolucionais até o bottleneck (e como ela é espelhada, o decoder também). Experimente variar a profundidade da rede (ou seja, o numero de blocos convolucionais), com 3 blocos e 7 blocos, por exemplo. Qual é o impacto no resultado? Reflita sobre como isso se relaciona com a teoria vista em sala de aula. Analise o impacto no desempenho e na quantidade de parâmetros.

6. **Summary da Arquitetura**:  
   Utilize a função `summary()` para exibir a quantidade de parâmetros do seu modelo.

7. **Escolha do Otimizador e Função de Perda**:  
   Escolha um otimizador e a função de perda apropriada para a tarefa de segmentação semântica.

8. **Função de Treinamento**:  
   Crie uma função de treinamento. Salve as métricas durante o treinamento em um dicionário `history`, contendo os valores de `train_loss`, `val_loss`, `train_acc` e `val_acc` a cada época.

9. **Treinamento do Modelo**:  
   Treine o modelo, a cada 5 ou 10 epochs, faça o **plot** de uma imagem contendo a **imagem original**, a **máscara verdadeira (ground truth)** e a **máscara predita pelo modelo**. Isso ajudará a visualizar a evolução da segmentação.

10. **Data Augmentation**:  
    Implemente um procedimento de data augmentation e treine novamente a melhor configuração observada. Houve ganho no desempenho? Reflita sobre os motivos.


11. **Preparação de uma Apresentação**:  
    Ao final, prepare uma apresentação resumindo os passos seguidos, resultados obtidos, gráficos de perdas e acurácia, e discussões sobre o desempenho do modelo. Lembre de fundamentar a discussão com os aspectos teoricos vistos em sala de aula.


### **Pontos Importantes:**

- Escolher adequadamente o tamanho do BATCH, Loss Function e Otimizador e saber o motivo de cada escolha;
- Monitore o uso das GPUs, o kaggle te informa quantidade de tempo disponível, mas o colab não;
- Observe as classes com mais erros por parte do modelo;
- Adicione gráficos de perda e acurácia na sua apresentação;
- Coloque imagens das predições do modelo;
- Use **Pytorch !!!**.



In [6]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import random
import torch
import shutil

In [7]:
class PreprocessDataset(Dataset):
    def __init__(self, dir_dataset, df_labels, dir_new_dataset, with_brightness_contrast=True, var_gaussian=10, amount = 0.02, prob_train=0.4, prob_others=0.1, partition=3): 
        self.with_brightness_contrast = with_brightness_contrast
        self.var_gaussian = var_gaussian
        self.amount = amount
        self.dir_dataset = dir_dataset
        self.prob_train = prob_train
        self.prob_others = prob_others
        self.dir_new_dataset = dir_new_dataset
        self.partition = partition
        self.rgb_to_index, self.index_to_label = self.dict_labels(df_labels)
        self.df_labels = df_labels
        self.dim_images = None
        self.num_images_train = None
        self.num_images_val = None
        self.num_images_test = None
        self.image_list = [] 
        self.label_list = [] 
        self.num_classes = len(self.df_labels)

    def __len__(self):
        return self.num_images_train + self.num_images_val + self.num_images_test
    
    def __getitem__(self, idx):
        # Carregar a imagem e o rótulo baseado no índice `idx`
        img_path = self.image_list[idx]
        lbl_path = self.label_list[idx]
        
        # Carregar a imagem e o rótulo do disco
        img = cv2.imread(img_path)
        lbl = cv2.imread(lbl_path)
        
        # Normalizar a imagem (opcional)
        img = img / 255.0
        
        # Converter os dados para tensores
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)  # De (H, W, C) para (C, H, W)
        lbl = torch.tensor(lbl, dtype=torch.long)
        
        return img, lbl

    def create_or_reset_directory(self, path):
        # Se o diretório já existir, esvaziar
        if os.path.exists(path):
            shutil.rmtree(path)  # Remove todo o conteúdo
        os.makedirs(path)  # Cria o diretório novamente
        
    def load_images_from_folder(self):
        # Criar ou esvaziar as subpastas para train, val, test
        sets = ['train', 'train_labels', 'val', 'val_labels', 'test', 'test_labels']
        
        for set_type in sets:
            dir_path = os.path.join(self.dir_new_dataset, set_type)
            self.create_or_reset_directory(dir_path)

        # Definir os caminhos originais dos datasets
        dirs = {set_type: os.path.join(self.dir_dataset, set_type) for set_type in sets}
        
        # Carregar e processar as imagens de cada conjunto
        for set_type in ['train', 'val', 'test']:
            images_dir = dirs[set_type]
            labels_dir = dirs[set_type + '_labels']
            num_images, dim_images = self.load_images_and_labels(images_dir, labels_dir, set_type)
            self.dim_images = dim_images
            if set_type == 'train':
                self.num_images_train = num_images
            elif set_type == 'val':
                self.num_images_val = num_images
            else:
                self.num_images_test = num_images
            
    def load_images_and_labels(self, images_dir, labels_dir, set_type):
        # Diretórios onde as imagens e rótulos processados serão salvos
        processed_dir = os.path.join(self.dir_new_dataset, set_type)
        processed_label_dir = os.path.join(self.dir_new_dataset, f"{set_type}_labels")
        count_images = 0   
        
        # Listar arquivos nos diretórios
        image_files = sorted(os.listdir(images_dir))
        label_files = sorted(os.listdir(labels_dir))
        
        prob_to_transform = self.prob_train if set_type == 'train' else self.prob_others
        img_lbl_files = zip(image_files, label_files)
        
        for img_name, lbl_name in img_lbl_files:
            transform_img = random.choices([True, False], weights=[prob_to_transform, 1 - prob_to_transform], k=1)[0]
            
            # Definir caminho para a imagem e rótulo
            img_path = os.path.join(images_dir, img_name)
            lbl_path = os.path.join(labels_dir, lbl_name)

            # Carregar imagem e rótulo
            img = cv2.imread(img_path)
            lbl = cv2.imread(lbl_path)

            # Reduzir a resolução da imagem
            img = img[::self.partition, ::self.partition, :]
            lbl = lbl[::self.partition, ::self.partition, :]

            # Converter de BGR para RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            lbl = cv2.cvtColor(lbl, cv2.COLOR_BGR2RGB)

            # Aplicar Gaussian blur
            img = cv2.GaussianBlur(img, (3, 3), 0)

            # Transformação aleatória
            if transform_img:
                img_transform = img.copy()
                lbl_transform = lbl.copy()
                img_transform, lbl_transform, trans = self.apply_random_transformation(img, lbl)

                # Salvar imagem e rótulo transformado
                count_images += 1
                processed_img_path = os.path.join(processed_dir, f"{img_name}_{trans}.png")
                processed_lbl_path = os.path.join(processed_label_dir, f"{lbl_name}_{trans}.png")
                cv2.imwrite(processed_img_path, img_transform)
                cv2.imwrite(processed_lbl_path, lbl_transform)

                # Adicionar as imagens e rótulos transformados às listas
                self.image_list.append(processed_img_path)
                self.label_list.append(processed_lbl_path)
                
            # Salvar imagem e rótulo
            count_images += 1
            processed_img_path = os.path.join(processed_dir, img_name)
            processed_lbl_path = os.path.join(processed_label_dir, lbl_name)

            cv2.imwrite(processed_img_path, img)
            cv2.imwrite(processed_lbl_path, lbl)

            # Adicionar as imagens e rótulos processados às listas
            self.image_list.append(processed_img_path)
            self.label_list.append(processed_lbl_path)

        return count_images, img.shape

    def apply_random_transformation(self, img, lbl):
        if self.with_brightness_contrast:
            random_transform = random.randint(0, 9)
        else:
            random_transform = random.randint(0, 4)
        if random_transform == 0:
            arg_1, arg_2 = self.augment(img, lbl, "rotation")
            return arg_1, arg_2, "rotation"
        elif random_transform == 1:
            arg_1, arg_2 = self.augment(img, lbl, "flip")
            return arg_1, arg_2, "flip"
        elif random_transform == 2:
            arg_1, arg_2 = self.augment(img, lbl, "flip_rotation")
            return arg_1, arg_2, "flip_rotation"
        elif random_transform == 3:
            arg_1, arg_2 = self.equalize_histogram(img, lbl)
            return arg_1, arg_2, "equalize_histogram"
        elif random_transform == 4:
            arg_1, arg_2 = self.add_noise(img, lbl, noise_type="gaussian")
            return arg_1, arg_2, "gaussian_noise"
        elif random_transform == 5:
            arg_1, arg_2 = self.add_noise(img, lbl, noise_type="salt_pepper")
            return arg_1, arg_2, "sal_pepper_noise"
        elif random_transform == 6:
            arg_1, arg_2 = self.adjust_brightness_contrast(img, lbl, brightness=0, contrast=50)
            return arg_1, arg_2, "b_c_0_50"
        elif random_transform == 7:
            arg_1, arg_2 = self.adjust_brightness_contrast(img, lbl, brightness=-10, contrast=30)
            return arg_1, arg_2, "b_c_-10_30"
        elif random_transform == 8:
            arg_1, arg_2 = self.adjust_brightness_contrast(img, lbl, brightness=60, contrast=60)
            return arg_1, arg_2, "b_c_60_60"
        else:
            arg_1, arg_2 = self.adjust_brightness_contrast(img, lbl, brightness=-20, contrast=0)
            return arg_1, arg_2, "b_c_-20_0"

    def rgb_to_label(self, mask):
        unique_colors = np.unique(mask.reshape(-1, mask.shape[2]), axis=0)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int32)
        for color in unique_colors:
            color_tuple = tuple(color)
            if color_tuple in self.rgb_to_index:
                label = self.rgb_to_index[color_tuple]
                matches = np.all(mask == color, axis=-1)
                label_mask[matches] = label
        return label_mask

    def dict_labels(self, df_labels):
        rgb_to_index = {}
        index_to_label = {}
        for count, row in df_labels.iterrows():
            color = (row['r'], row['g'], row['b'])
            label = row['name']
            rgb_to_index[color] = count
            index_to_label[count] = label
        return rgb_to_index, index_to_label

    def augment(self, img, lbl, transform):
        if transform == "rotation":
            rotation = cv2.getRotationMatrix2D((img.shape[1] // 2, img.shape[0] // 2), 180, 1)
            rotated_image = cv2.warpAffine(img, rotation, (img.shape[1], img.shape[0]))
            rotated_label = cv2.warpAffine(lbl, rotation, (lbl.shape[1], lbl.shape[0]), flags=cv2.INTER_NEAREST)
            return rotated_image, rotated_label
        elif transform == "flip":
            flipped_image = cv2.flip(img, 1)
            flipped_label = cv2.flip(lbl, 1)
            return flipped_image, flipped_label
        elif transform == "flip_rotation":
            flipped_image = cv2.flip(img, 0)
            flipped_label = cv2.flip(lbl, 0)
            return flipped_image, flipped_label
        
   
    def equalize_histogram(self, img, lbl):
        img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
        return img, lbl
        
    def adjust_brightness_contrast(self, img, lbl, brightness=0, contrast=0):
        img = np.int16(img)
        img = img * (contrast / 127 + 1) - contrast + brightness
        img = np.clip(img, 0, 255)
        img = np.uint8(img)
        return img, lbl
    
    def add_noise(self, img, lbl, noise_type="gaussian"):
        if noise_type == "gaussian":
            mean = 0
            var = self.var_gaussian
            sigma = var ** 0.5
            gauss = np.random.normal(mean, sigma, img.shape)
            noisy = img + gauss
            noisy = np.clip(noisy, 0, 255)
            return noisy, lbl
    
        elif noise_type == "salt_pepper":
            s_vs_p = 0.5
            amount = self.amount
            out = np.copy(img)
    
            # Salt mode
            num_salt = np.ceil(amount * img.size * s_vs_p)
            coords = [np.random.randint(0, i, int(num_salt)) for i in img.shape]
            out[tuple(coords)] = 255
    
            # Pepper mode
            num_pepper = np.ceil(amount * img.size * (1.0 - s_vs_p))
            coords = [np.random.randint(0, i, int(num_pepper)) for i in img.shape]
            out[tuple(coords)] = 0
    
            return out, lbl

In [8]:
print("Cuda is available: ", torch.cuda.is_available())

# Definindo os caminhos para cada conjunto
dir_dataset = "data\CamVid"

class_dict = os.path.join(dir_dataset, 'class_dict.csv')
df_labels = pd.read_csv(class_dict)

Cuda is available:  True


<>:4: SyntaxWarning: invalid escape sequence '\C'
<>:4: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Isas_\AppData\Local\Temp\ipykernel_7668\1077995894.py:4: SyntaxWarning: invalid escape sequence '\C'
  dir_dataset = "data\CamVid"


In [9]:
# Instanciar a classe PreprocessDataset
partition = 3  # Reduzir o tamanho da imagem por um fator de 3
prob_train = 0.35  # Probabilidade de aplicar transformações no conjunto de treino
prob_others = 0.15  # Probabilidade de aplicar transformações nos conjuntos de validação e teste
dir_new_dataset = "processed_data" # Diretório onde os conjuntos processados serão salvos
with_brightness_contrast = True # Aplicar transformações de brilho e contraste
var_gaussian = 20 # Variância do ruído gaussiano
amount = 0.02 # Quantidade de ruído sal e pimenta
preprocessor = PreprocessDataset(dir_dataset, df_labels, dir_new_dataset, with_brightness_contrast, var_gaussian, amount, prob_train, prob_others, partition)

# Carregar e processar as imagens e rótulos
preprocessor.load_images_from_folder()

# Número de imagens em cada conjunto
print("Número de imagens de treino: ", preprocessor.num_images_train)
print("Número de imagens de validação: ", preprocessor.num_images_val)
print("Número de imagens de teste: ", preprocessor.num_images_test)
print("Dimensões das imagens: ", preprocessor.dim_images)
print("Número de classes: ", preprocessor.num_classes)

Número de imagens de treino:  507
Número de imagens de validação:  115
Número de imagens de teste:  266
Dimensões das imagens:  (240, 320, 3)
Número de classes:  32


In [ ]:
# Instanciar a classe PreprocessDataset
partition = 3  # Reduzir o tamanho da imagem por um fator de 3
prob_train = 0.25  # Probabilidade de aplicar transformações no conjunto de treino
prob_others = 0.10  # Probabilidade de aplicar transformações nos conjuntos de validação e teste
dir_new_dataset = "processed_data_0" # Diretório onde os conjuntos processados serão salvos
with_brightness_contrast = True # Aplicar transformações de brilho e contraste
var_gaussian = 20 # Variância do ruído gaussiano
amount = 0.02 # Quantidade de ruído sal e pimenta
preprocessor = PreprocessDataset(dir_dataset, df_labels, dir_new_dataset, with_brightness_contrast, var_gaussian, amount, prob_train, prob_others, partition)

# Carregar e processar as imagens e rótulos
preprocessor.load_images_from_folder()

# Número de imagens em cada conjunto
print("Número de imagens de treino: ", preprocessor.num_images_train)
print("Número de imagens de validação: ", preprocessor.num_images_val)
print("Número de imagens de teste: ", preprocessor.num_images_test)
print("Dimensões das imagens: ", preprocessor.dim_images)
print("Número de classes: ", preprocessor.num_classes)

In [10]:
# (240, 320) block 1
# (120, 160) block 2
# (60, 80) block 3
# (30, 40) block 4
# (15, 20) block 5

In [11]:
class UNet(nn.Module):
    def __init__(self, image_dim, n_channels=3, n_classes=32, depth=5, conv_kernel_size=3, conv_stride=1, conv_padding=1, pool_kernel_size=2, pool_stride=2, pool_padding=0, transpose_kernel_size=3, transpose_stride=2, transpose_padding=1):
        super(UNet, self).__init__()

        self.image_dim = image_dim  # Dimensões da imagem de entrada (C, H, W)
        self.depth = depth
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.conv_kernel_size = conv_kernel_size
        self.conv_stride = conv_stride
        self.conv_padding = conv_padding
        self.pool_kernel_size = pool_kernel_size
        self.pool_stride = pool_stride
        self.pool_padding = pool_padding
        self.transpose_kernel_size = transpose_kernel_size
        self.transpose_stride = transpose_stride
        self.transpose_padding = transpose_padding

        # Encoder
        self.encoders = nn.ModuleList([self.conv_block(self.n_channels if i == 0 else 64 * (2 ** (i-1)), 64 * (2 ** i)) for i in range(depth)])
        self.pool = nn.MaxPool2d(kernel_size=self.pool_kernel_size, stride=self.pool_stride, padding=self.pool_padding)

        # Bottleneck
        self.bottleneck = self.conv_block(64 * (2 ** (depth-1)), 64 * (2 ** depth))

        # Decoder
        self.decoders = nn.ModuleList([self.conv_transpose(64 * (2 ** (i+1)), 64 * (2 ** i)) for i in range(depth-1, -1, -1)])

        # Final conv layer
        self.final_conv = nn.Conv2d(64, n_classes, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        # Camada convolucional com normalização e função de ativação; 2 vezes
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=self.conv_kernel_size, stride=self.conv_stride, padding=self.conv_padding),
            nn.BatchNorm2d(out_channels),  # Normalização para acelerar o treinamento
            nn.ReLU(inplace=True),  # Função de ativação (zera os valores negativos)
            nn.Conv2d(out_channels, out_channels, kernel_size=self.conv_kernel_size, stride=self.conv_stride, padding=self.conv_padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def crop(self, encoder_feature, decoder_feature):
        _, _, h, w = decoder_feature.size()
        encoder_feature = F.interpolate(encoder_feature, size=(h, w), mode='bilinear', align_corners=False)  # Redimensiona a feature map do encoder
        return encoder_feature

    def conv_transpose(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=self.transpose_kernel_size, stride=self.transpose_stride, padding=self.transpose_padding),
            self.conv_block(out_channels, out_channels)
        )

    def forward(self, x):
        encoders_features = []

        # Encoder pass
        for encoder in self.encoders:
            x = encoder(x)
            encoders_features.append(x)
            x = self.pool(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Decoder pass
        for i, decoder in enumerate(self.decoders):
            encoder_feature = encoders_features[-(i+1)]
            encoder_feature = self.crop(encoder_feature, x)  # Aplica o crop nas feature maps
            x = torch.cat([encoder_feature, x], dim=1)  # Concatena encoder com decoder
            x = decoder(x)

        # Final convolution
        x = self.final_conv(x)
        return x

In [12]:
# Criando o DataLoader
batch_size = 4 # Tamanho do batch: número de amostras que serão carregadas a cada iteração
train_loader = DataLoader(
    preprocessor,  # O dataset que criamos
    batch_size=batch_size,  
    shuffle=True,  # Embaralha os dados a cada epoch
    num_workers=4  # Quantos subprocessos usar para carregar os dados (ajuste para sua máquina)
)

In [13]:
# Instanciando o modelo U-Net com as dimensões das imagens
model = UNet(image_dim=(3, 240, 320), n_channels=3, n_classes=32)

# Configurando o dispositivo (GPU, se disponível)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo: {device}")
model = model.to(device)  # Mover o modelo para o dispositivo

# Testando com um batch de dados do DataLoader
for images, labels in train_loader:
    # Move os dados para o dispositivo (GPU/CPU)
    images = images.to(device)
    labels = labels.to(device)
    
    # Passa as imagens pelo modelo U-Net
    output = model(images)
    
    # Exibe as dimensões das imagens, labels e da saída do modelo
    print(f"Imagens: {images.shape}")
    print(f"Labels: {labels.shape}")
    print(f"Saída do modelo: {output.shape}")
    
    # Quebrar após o primeiro batch, apenas para teste
    break

Dispositivo: cuda
